In [1]:
# %%sh

import os
import sys
import re
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import openpyxl
import pandas as pd
import math
import itertools
import matplotlib.patches as patches
import seaborn as sns
from decimal import Decimal
import glob
import csv
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import sklearn.svm 
import sklearn.metrics
from sklearn.feature_selection import RFE
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers


def select_features(X, y, topN):
    clf = sklearn.svm.SVC(kernel='linear')    
    clf.fit(X, y)
    coef_list = clf.coef_[0][:]
    coef_list =abs(coef_list)
    top_idx = (np.argsort(coef_list)[-topN:]).tolist()
    top_idx.reverse()
    return top_idx, clf.coef_[0][top_idx]


def svmClassify(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)
    clf = sklearn.svm.SVC(gamma=0.001, class_weight='balanced', C=1.0, probability=True, random_state=1)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_train)      
    train_accuracy = sklearn.metrics.accuracy_score(y_train, y_pred)
    
    y_pred = clf.predict(X_test)
    probas = clf.predict_proba(X_test)    
    test_accuracy = sklearn.metrics.accuracy_score(y_test, y_pred)
    return train_accuracy,  test_accuracy

arch = "P100"
dataName= 'perf_transfer_%s.csv'%arch
dataX = pd.read_csv(dataName, sep=',' )
y = dataX.loc[:,"Type"]
y = [int(i=='risky') for i in y]
X = dataX.iloc[:, 4:]
X_std =  StandardScaler().fit_transform(X.values)

top_idx, coef_list = select_features(X_std, y, 20)
accuracy_nn = []
print("topN", "train_accuracy", "val_accuracy",  "val_accuracy")
for topN in range(2, 21):
    topN_idx = top_idx[:topN]
    X_input = X_std[:, topN_idx]
    train_accuracy, test_accuracy = svmClassify(X_input, y)
    accuracy_nn.append([topN, train_accuracy, test_accuracy])
    print(topN, train_accuracy, test_accuracy)

accuracy_nn = np.asarray(accuracy_nn)
accuracy_df = pd.DataFrame(accuracy_nn, columns=["topN", "train_accuracy",  "test_accuracy"])
accuracy_df.to_csv(dataName.split('.')[0] + "_svmAccuray.csv", index = False)

#%%





/software/anaconda3/5.1.0/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


topN train_accuracy val_accuracy val_accuracy
2 0.7972222222222223 0.7833333333333333
3 0.8018518518518518 0.7916666666666666
4 0.7944444444444444 0.7805555555555556
5 0.7944444444444444 0.7805555555555556
6 0.7916666666666666 0.7833333333333333
7 0.8472222222222222 0.8527777777777777
8 0.8425925925925926 0.8416666666666667
9 0.8407407407407408 0.8416666666666667
10 0.8611111111111112 0.8694444444444445
11 0.8564814814814815 0.8638888888888889
12 0.8546296296296296 0.8666666666666667
13 0.8546296296296296 0.8611111111111112
14 0.8731481481481481 0.8861111111111111
15 0.8722222222222222 0.8833333333333333
16 0.875 0.8833333333333333
17 0.9064814814814814 0.9138888888888889
18 0.8972222222222223 0.9083333333333333
19 0.8981481481481481 0.9083333333333333
20 0.9018518518518519 0.9111111111111111
